# PRAW Library
## Python Reddit API Wrapper

In [ ]:
# install library
# !pip install praw

In [ ]:
import praw
import pandas as pd
import networkx as nx
#import matplotlib.pyplot as plt

In [ ]:
clientid = "EppA5GzbdocKzA"
clientsecret = "PBMQFMsqWuXuqGak1qVNlhzyi4A"
password = 'password'
username = "Aidenberg"
user_agent = "testcript by /u/aiden"

In [ ]:
reddit = praw.Reddit(client_id=clientid, 
                     client_secret=clientsecret,
                     password= password, 
                     user_agent='testcript by /u/aiden',
                     username='Aidenberg')

In [ ]:
reddit.user.me()

### Play with PRAW

In [ ]:
# load submissions title
subreddit = reddit.subreddit('python')
hot_python = subreddit.hot(limit=5)
for submission in hot_python:
    print(submission.title)

In [ ]:
# load more information
hot_python = subreddit.hot(limit=5)
for submission in hot_python:
    print(submission.title,
          submission.ups,
          submission.downs,
          submission.visited)

In [ ]:
# What else we can do with praw
subreddit.subscribe()
subreddit.unsubscribe()

In [ ]:
# Comments
hot_python = subreddit.hot(limit=5)
for submission in hot_python:
    comments = submission.comments
    for comment in comments:
        print(20*'-')
        print(comment.body)

In [ ]:
# Comments with threads
# error occurs if thread is too long.

#try again with 'news' subreddit
hot_python = reddit.subreddit('python').hot(limit=5)
for submission in hot_python:
    comments = submission.comments
    for comment in comments:
        print(20*'-')
        print('Parent ID:', comment.parent())
        print('Comment ID:', comment.id)
        print(comment.body)

In [ ]:
hot_python = reddit.subreddit('news').hot(limit=1)
for submission in hot_python:
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        print(20*'-')
        print('Parent ID:', comment.parent())
        print('Comment ID:', comment.id)
        print(comment.body)

## Comment Network

In [ ]:
col_edge = ['parentID', 'commentID']
lst = []
hot_python = reddit.subreddit('news').hot(limit=1)
for submission in hot_python:
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        lst.append([comment.parent().author, comment.author])
            
df_edge = pd.DataFrame(lst, columns=col_edge)

In [ ]:
df_edge.head(5)

In [ ]:
col_node = ['userID']
df_node = pd.DataFrame(list(set(sum(lst, []))), columns=col_node)

In [ ]:
G = nx.Graph()

for index, row in df_node.iterrows():
    G.add_node(row['userID'])
for index, row in df_edge.iterrows():
    G.add_edges_from([(row['parentID'], row['commentID'])])

In [ ]:
#plt.figure(figsize=(25,25))
#options = {
#    'edge_color': '#FFDEA2',
#    'width': 1,
#    'with_labels': False,
#    'font_weight': 'regular',
#}
#
#pos = nx.kamada_kawai_layout(G)
#nx.draw(G, node_color = 'lightblue', node_size = [G.degree[i]*50 for i in G], pos=pos, **options)
#        
#ax = plt.gca()
#ax.collections[0].set_edgecolor("#555555") 
#plt.show()

<img src="comment_network.png">

### Collect node attributes

User Karma

User Subscription or active subreddits

Edge-wise attributes can be collected by considering common subreddits.


#### 1. Collect Karma

In [ ]:
# Karma can be collected by redditor names
user = reddit.redditor("bboe")
user.comment_karma, user.link_karma

In [ ]:
# Karma can be collected by reddior ids (reddit.get function)
# Multiple requests

subreddit = reddit.subreddit('redditdev')
user_ids = [user.id for user in subreddit.moderator()]
data = reddit.get('api/user_data_by_account_ids',
                  params={'ids': ','.join(user_ids)})
for id_, d in data.items():
    print('{name}: {comment_karma} {link_karma}'.format(**d))

#### 2. Collect Date the account created

In [ ]:
# When were the accounts created?
import time
for id_, d in data.items():
    print('{}: {}'.format(d.get("name"),
                          time.ctime(d.get("created_utc"))))

#### 4. Active subreddits per user

In [ ]:
#user = praw.models.Redditor(reddit, name="/u/fwork", _data=None)
comments_path ="user/bboe/comments" 
comments = praw.models.ListingGenerator(reddit, comments_path, params=None, limit=100)
comments = comments.__iter__()

In [ ]:
lst = []
for comment in comments:
    lst.append(comment.subreddit.display_name)

In [ ]:
from collections import Counter
Counter(lst).most_common()

## Subreddit Network

In [ ]:
## Creating a subreddit network
## connections among subreddits
## how to define edges?

In [ ]:
subreddit = reddit.subreddit("programming")

In [ ]:
#subreddit = reddit.subreddit('politics')
#   for submission in subreddit.submissions(1478592000, 1478678400):
#       print(submission.title)

##### Weight of edges can be assessed by common users

In [ ]:
# Consider 7 different subreddits
subreddit_list = ["programming", "python", "technology", "coding", "django", "philosophy", "PhilosophyofReligion"]
LST = []
for sr_list in subreddit_list:
    subreddit = reddit.subreddit(sr_list)
    lst = []
    for i in subreddit.hot(limit=500):
        lst.append(i.author)
    LST.append(lst)

In [ ]:
LST # returns Redditor object. Need to convert them into strings

In [ ]:
# LST is a list of redditor objects
# Changing them into user names
l = []; L = []
for i in range(0,7):
    for j in LST[i]:
        if j != None:
            l.append(j.name)
    L.append(l)
    l = []

In [ ]:
D = pd.DataFrame(columns = subreddit_list,
                 index = subreddit_list) # contingency matrix storing the number of common users

for i in range(0,7): # for loop for common users
    for j in range(0,7):
        if i < j:
            common_users = set(L[i]).intersection(L[j])
            print('Subreddits:({}, {}) \n Len:{} \n Users: {}'.format(subreddit_list[i], subreddit_list[j], 
                                                                      len(common_users), common_users))
            print(20*'-')
            
            D.at[subreddit_list[i],subreddit_list[j]] = D.at[subreddit_list[j],subreddit_list[i]] = len(common_users)

In [ ]:
D

##### Edges by X-post
We may also use X-posts. X-post is cross-post meaning it locates in multiple subreddits.

In [ ]:
subreddit_list = ["programming", "python", "technology", "coding", "django", "philosophy", "PhilosophyofReligion"]
LST = []
for sr_list in subreddit_list:
    subreddit = reddit.subreddit(sr_list)
    lst = []
    for i in subreddit.search("x-post", limit=500):
        lst.append(i.author)
    LST.append(lst)

In [ ]:
import re #library for regular expression

LST2 = []
for sr_list in subreddit_list:
    subreddit = reddit.subreddit(sr_list)
    
    lst = []
    for i in subreddit.search("x-post", limit=500):
        tmp = re.search(r'r\/(.*?(?=[\]\)\s\/\:]))', i.title)
        if tmp != None:
            lst.append(tmp.group(1))
            
    LST2.append(lst)

In [ ]:
LST2
# We can also make edge attributes of weights based on x-posts.